In [1]:
import os, cv2
import splitfolders
import shutil

In [2]:
# Define your input and output folder paths
input_folder = '/home/herve/agsolaire_ml_UNL/data'
output_folder = '/home/herve/agsolaire_ml_UNL/train_val_test_split'

# Check if the output folder exists and delete it if it does
try:
    shutil.rmtree(output_folder)
    print(f"Removed existing folder: {output_folder}")
except FileNotFoundError:
    print(f"Output folder not found, creating a new one: {output_folder}")

# Perform the train/validation/test split
splitfolders.ratio(
    input_folder, 
    output=output_folder, 
    seed=1337, 
    ratio=(0.8, 0.1, 0.1), 
    move=False
)

Removed existing folder: /home/herve/agsolaire_ml_UNL/train_val_test_split


In [3]:
from ultralytics import YOLO
import torch

print(torch.__version__)

model   = YOLO('yolo11s.pt')

results = model.train(
    data   = '/home/herve/agsolaire_ml_UNL/yolo_training.yaml',
    imgsz  = 640,
    epochs = 40,
    batch  = 16,
    name   = 'agsolaire_yolo_training',
    device = 0
   
)

model.val()

2.8.0+cu128
Ultralytics 8.3.199 🚀 Python-3.10.12 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7832MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/herve/agsolaire_ml_UNL/yolo_training.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=agsolaire_yolo_training6, nbs=64, nms=False, opset=None, optimize=False, optimizer=au

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x788720bb6fe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 